In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

## Import framework main libs

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic
import pandas as pd

## Import strategy from module

In [ ]:
from tmqralphas.strategy_dsp_bandpass import Strategy_DSP_BandPass

In [ ]:
ALPHA_CONTEXT = {
        'name': 'ES_NewFramework_DSP_BandPass', # Global alpha name, which be used for load/save from DB
        'context': { # Strategy specific settings
            # These settings only applycable for alphas derived from StrategyAlpha strategy 
            # StrategyAlpha - is a classic EXO/SmartEXO based alpha
            'index_name': 'US.ES_ContFutEOD',      # Name of EXO index to trade
            'costs_per_option': 3.0,
            'costs_per_contract': 3.0,
        },
        'wfo_params': {
            'window_type': 'expanding',  # Rolling window for IIS values: rolling or expanding
            'period': 'W',  # Period of rolling window 'M' - monthly or 'W' - weekly
            'oos_periods': 2,  # Number of months is OOS period
            'iis_periods': 12,
            # Number of months in IIS rolling window (only applicable for 'window_type' == 'rolling')
        },
        'wfo_optimizer_class': OptimizerBase, 
        'wfo_optimizer_class_kwargs': {
            'nbest_count': 3,
            'nbest_fitness_method': 'max',
            'population_size': 10, 
            'number_generations': 3, 
            'rand_seed': 1, # Uncomment this parameter to make genetic results repeatable
        },
        'wfo_opt_params': [
            ('direction', [-1]),
            ('Filter order',  [3, 5, 4]),
            ('Filter start freq', [0.5366, 0.4979, 0.4491, 0.4675, 0.5364]), # 0 > f < 1
            ('Filter stop freq', [0.5511, 0.6529, 0.5595, 0.5406, 0.6116]), # 0 > f < 1
            ('Filter sigma', [0.7413, 0.71, 0.7967, 0.7616, 0.7488]),
            ('Rule index', [3, 2]),
        ],
        'wfo_members_count': 1,
        'wfo_costs_per_contract': 0.0,
        'wfo_scoring_type': 'modsharpe'
    }

# Run the alpha

In [ ]:
# DataManager is a core class of the framework
dm = DataManager()

# Init alpha class and run
alpha = Strategy_DSP_BandPass(dm, **ALPHA_CONTEXT)    

In [ ]:
#alpha.setup()
alpha.run()

In [ ]:
alpha.stats['series']['equity'].plot()

# Genetic optimizer swarms view

In [ ]:
from tmqrstrategy.reports import GeneticSwarmViewer

#
# Replace compatible opt_params of the alpha here
#
'''opt_params = [
            ('direction', [1]),
            ('Filter order',  [2]),
            ('Filter start freq', [0.8081, 0.8077, 0.809, 0.8211, 0.8085]), # 0 > f < 1
            ('Filter stop freq', [0.4902, 0.4903, 0.4902, 0.4894, 0.4902]), # 0 > f < 1
            ('Filter sigma', [0.5088, 0.5169, 0.5184, 0.646, 0.6539]),
            ('Rule index', [2]),
        ]
'''
#
# OR JUST USE ALPHA_CONTEXT
#
# !!! Comment next line if you want to use manual opt_params settings above
opt_params = ALPHA_CONTEXT['wfo_opt_params']

gsv = GeneticSwarmViewer(alpha, opt_params,                          
                         oos_period_ratio=0.5,    # OOS period as of % of full history
                         nbest_count=1,         # Number of best swarm members to select
                         population_size=10,     # Number of randomly selected swarm members per population
                         number_generations=30)   # Number of generations of population

gsv.run()

# Set report graphs size
figsize(7, 3)

# Show report
gsv.report()

In [ ]:
gsv.df_full_swarm.shape

In [ ]:
for i in range(100):
    print(uniform(0,1))